# Amazon Basics 7: Identity and Access Management

## Usage Notes

Amazon Identity and Access Management (IAM) is designed to allow you to control access to Amazon resources. The idea is to allow you to restrict access to entities that are created in Amazon, such as users and objects.

## Notebook Imports

In [ ]:
from __future__ import print_function
from aws_base import *
import json
import os
import subprocess

## Update Your User

These notebooks make calls on behalf of the user set via `aws configure`. These calls include creating EC2 instances, list IAM roles, manage security groups, and to list the names of your EC2 private keys (to confirm that any private key you specify is valid in your region).

In order to allow these notebooks sufficient privileges to do their work, it's probably easiest to attach policies which allow us to know about IAM roles and the ability to create and manage EC2 instances.

https://console.aws.amazon.com/iam/home#users

Select the user that you created in the previous notebook and click on the **Add Permissions** button. On the permissions screen, select **Attach existing policies directly** and add `IAMReadOnlyAccess` and `AmazonEC2FullAccess`.

In more advanced environments where there are many users, it is more desirable to attach those policies (or similar policies) to groups and assign your current AWS user to those groups. However, while this is a best practice, doing so is not a requirement for these exercises.

https://console.aws.amazon.com/iam/home#groups

### Confirm IAMReadOnlyAccess

The following is a spot test which prints the user name of the user whose credentials were stored via `aws configure`. However, it will only work if you have the equivalent of the `IAMReadOnlyAccess` privilege.

In [ ]:
user_name = None

try:
    user_json = aws('iam', 'get-user')
    user = user_json['User']
    user_name = user['UserName']
except:
    print('Insufficient IAM privileges')

user_name

### Confirm AmazonEC2FullAccess, Part 1

The following is a spot test which retrieves the list of regions available for use in Amazon Web Service requests. It will only work if you have some level of EC2 access, though since it's an API read, it didn't actually need `AmazonEC2FullAccess` capabilities.

In [ ]:
aws_regions_json = aws('ec2', 'describe-regions')
aws_regions = aws_regions_json['Regions']
region_names = [aws_region['RegionName'] for aws_region in aws_regions]

Also to confirm your current settings, we'll see if the region you have set via `aws configure` matches one of the regions returned by this API read.

In [ ]:
assert region in region_names

### Confirm AmazonEC2FullAccess, Part 2

To truly test this out, the following section will attempt to identify the image ID for an Ubuntu 14.04 AMI based on your current region.

Run the following command, which performs a dry-run (permissions check, parameter validation) of EC2 instance creation, to see if you are allowed to run the default Ubuntu image. This will validate the `AmazonEC2FullAccess` policy.

In [ ]:
!aws ec2 run-instances --dry-run \
    --image-id={get_default_image_id('hvm')} --instance-type t2.nano

## Confirm IAM EC2 Policy

### Confirm IAM Role

Roles are a way to grant privileges to the instance itself through using Amazon's Identity and Access Management (IAM) roles. They are used to allow specific users to elevate their privileges for your account's resources or to allow an EC2 instance to have specific privileges on initialization.

* https://console.aws.amazon.com/iam/home#roles

For our purposes, we want to allow our EC2 instances to download from S3 buckets without having to specify user credentials. You can achieve this by creating an **AWS Service Role** for **Amazon EC2** with the `AmazonS3ReadOnlyAccess` policy attached. You can also give more permissions to the role, should you believe them to be necessary for what you're doing with the EC2 instance.

Specify the name of the role that you created which has the minimum capabilities noted above.

In [ ]:
iam_role_name = 'TRAIN_EC2_DefaultRole'

The script below will retrieve the IAM Instance Profile that was created at the same time as your role so that it can use it when creating the EC2 instance. More information on instance profiles is available in the AWS documentation.

http://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-ec2_instance-profiles.html

In [ ]:
instance_profile_arn = None
iam_role_arn = None

if user_name is not None:
    instance_profile_json = aws(
        'iam', 'get-instance-profile', '--instance-profile-name', iam_role_name)

    instance_profile = instance_profile_json['InstanceProfile']

    instance_profile_arn = instance_profile['Arn']
    iam_role_arn = instance_profile['Roles'][0]['Arn']

[instance_profile_arn, iam_role_arn]

### Create IAM Policy

You will need to make sure that the current user has the ability to use this instance profile by making sure that the user has the ability to pass your declared role to the instance.

https://console.aws.amazon.com/iam/home#policies

This is how such a policy would be structured. You can take the output that is displayed and copy it to a new policy after replacing all the single quotes (the default way Python writes strings) and replacing them with double quotes.

In [ ]:
if user_name is not None:
    policy_info = {
        'Version': '2012-10-17',
        'Statement': [
            {
                'Effect': 'Allow',
                'Action': [ 'iam:PassRole' ],
                'Resource': [ str(iam_role_arn) ]
            },
            {
                'Effect': 'Allow',
                'Action': [ 'iam:AddRoleToInstanceProfile' ],
                'Resource': [ str(instance_profile_arn) ]
            }
        ]
    }

    !echo '{json.dumps(policy_info, indent=4)}'

To create this policy, navigate to the **Policies** section, click on the **Create Policies** button, click on the **Create Your Own Policy** button, give it any name and description you want (``Train_EC2_PassRole`` is what we'll be using as an example), and then paste in the above text.

Once that's done, attach your policy to your user. Click on the **Users** section, then select your user, click on the **Add Permissions** button, select the **Attach existing policies directly** option, then choose the policy that you created in the previous step.

### Confirm IAM Role Passing

After you've attached your policy to a user or group, run the following command, which performs a dry-run (permissions check, parameter validation) of EC2 instance creation, to see if you are allowed to run an instance using the instance profile associated with this IAM role.

In [ ]:
!aws ec2 run-instances --dry-run \
    --image-id={get_default_image_id('hvm')} --instance-type t2.nano \
    --iam-instance-profile Arn=$instance_profile_arn

If you're curious how to generate the policy through the Amazon GUI rather than simply copy-replace-paste something that was generated for you by a Python script, you can do the following:

* Choose the Create Policy option
* Follow the Policy Generator wizard
* Ensure that this policy allows assuming the role
    * Choose the AWS Identity and Access Management service
    * Select the PassRole action
    * Specify the role ARN above
    * Select Add Statement
* Ensure that this policy allows adding this role to an instance profile
    * Choose the AWS Identity and Access Management service
    * Select the AddRoleToInstanceProfile action
    * Specify the instance profile ARN above
    * Select Add Statement

## Convert Notebook to Script

The following cell will use `jupyter nbconvert` to build an `aws_iam.py` which will be used in future notebooks in this series.

In [ ]:
%%javascript
var script_file = 'aws_iam.py';

var notebook_name = window.document.getElementById('notebook_name').innerHTML;
var nbconvert_command = 'jupyter nbconvert --stdout --to script ' + notebook_name;

var grep_command = "grep -v '^#' | grep -v -F get_ipython | sed '/^$/N;/^\\n$/D'";
var command = '!' + nbconvert_command + ' | ' + grep_command + ' > ' + script_file;

if (Jupyter.notebook.kernel) {
    Jupyter.notebook.kernel.execute(command);
}